In [ ]:
import numpy as np
from scipy.stats.qmc import Sobol
from pydantic import BaseModel, ValidationError
from typing import Type, Any, List, Dict

In [ ]:
class SobolSampler:
    def __init__(self, pydantic_class: Type[BaseModel], dimensions: Dict[str, List[float]]):
        self.pydantic_class = pydantic_class
        self.dimensions = dimensions
        self.dimension_names = list(dimensions.keys())
        self.lower_bounds = np.array([v[0] for v in dimensions.values()])
        self.upper_bounds = np.array([v[1] for v in dimensions.values()])
        self.d = len(dimensions)
        self.sampler = Sobol(d=self.d, scramble=True)

    def sample(self, n_samples: int) -> List[BaseModel]:
        samples = self.sampler.random(n_samples)
        scaled_samples = self.lower_bounds + (self.upper_bounds - self.lower_bounds) * samples
        instances = []

        for sample in scaled_samples:
            data = {self.dimension_names[i]: sample[i] for i in range(self.d)}
            try:
                instance = self.pydantic_class(**data)
                instances.append(instance)
            except ValidationError as e:
                print(f"Validation error for sample {data}: {e}")

        return instances

In [ ]:
# Example usage:

class ExampleModel(BaseModel):
    x: float
    y: float

In [ ]:
dimensions = {
    'x': [0.0, 1.0],
    'y': [0.0, 1.0]
}

In [ ]:
sampler = Sampler(ExampleModel, dimensions)

In [ ]:
samples = sampler.sample(10)

for s in samples:
    print(s)

In [ ]:
more_samples = sampler.sample(10)

for s in more_samples:
    print(s)

In [ ]:
sampler.dimensions